# Example for:
Transfer Learning Empirical Experiment from all other classes of FashionMNIST to shoes (Sandal, Sneaker, Ankle boot) classes in FashionMNIST

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 6
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64, # num channels for CNN
      'two_linear_layers': False,
      'hidden_dim_lin': 128,  # if two_linear_layers == True
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 2, 4, 6, 7, 9],
      'fine_tune_classes': [3, 5, 8],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.FashionMNIST, datasets.FashionMNIST, root_dir, transform=transform)## Pretraining

In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

37800

## Pretraining

In [6]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [7]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 39.97%
Validation Accuracy: 39.19%
Epoch: 1 	Training Accuracy: 67.00%
Validation Accuracy: 66.95%
Epoch: 2 	Training Accuracy: 70.94%
Validation Accuracy: 70.86%
Epoch: 3 	Training Accuracy: 73.20%
Validation Accuracy: 72.67%
Epoch: 4 	Training Accuracy: 74.82%
Validation Accuracy: 74.10%
Epoch: 5 	Training Accuracy: 76.53%
Validation Accuracy: 75.43%
Epoch: 6 	Training Accuracy: 75.94%
Validation Accuracy: 75.64%
Epoch: 7 	Training Accuracy: 78.08%
Validation Accuracy: 77.57%
Epoch: 8 	Training Accuracy: 79.89%
Validation Accuracy: 78.52%
Epoch: 9 	Training Accuracy: 77.29%
Validation Accuracy: 76.74%
Epoch: 10 	Training Accuracy: 80.01%
Validation Accuracy: 79.17%
Epoch: 11 	Training Accuracy: 80.18%
Validation Accuracy: 79.45%
Epoch: 12 	Training Accuracy: 81.11%
Validation Accuracy: 80.52%
Epoch: 13 	Training Accuracy: 82.89%
Validation Accuracy: 81.67%
Epoch: 14 	Training Accuracy: 82.34%
Validation Accuracy: 82.31%
Epoch: 15 	Training Accuracy: 83.30

In [8]:
torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_cifar_arc/pretrained_model_random.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_cifar_arc/pretrained_model_random.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.3160, Accuracy: 3695.0/4200 (88%)

              precision    recall  f1-score   support

     Class 0       0.85      0.86      0.86       603
     Class 1       0.99      0.98      0.99       645
     Class 2       0.88      0.75      0.81       592
     Class 3       0.81      0.87      0.84       600
     Class 4       0.70      0.76      0.73       588
     Class 5       0.97      0.96      0.96       581
     Class 6       0.96      0.97      0.97       591

    accuracy                           0.88      4200
   macro avg       0.88      0.88      0.88      4200
weighted avg       0.88      0.88      0.88      4200



0.8797619047619047

In [16]:
# save model for later use
foldername = "pretrained_models/pretrained_normal_classes_new"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 1.0, Lr: 0.001, Repeat: 0


KeyboardInterrupt: 

In [ ]:
print(baselines_results)

[{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.84, 'test_acc': 0.7092666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.88, 'test_acc': 0.7712333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.84, 'test_acc': 0.7833}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.88, 'test_acc': 0.7267666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8, 'test_acc': 0.7448333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.84, 'test_acc': 0.6792}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.76, 'test_acc': 0.8136}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.72, 'test

In [20]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results/cifarstyle_baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_random.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [11]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1.0]

In [12]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9920, Test Accuracy: 0.9919

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Early stopping invoked.
Training Accuracy: 0.9881, Test Accuracy: 0.9883

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.9899, Test Accuracy: 0.9901

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Early stopping invoked.
Training Accuracy: 0.9892, Test Accuracy: 0.9893

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Training Accuracy: 0.9900, Test Accuracy: 0.9900

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.9913, Test Accuracy: 0.9914

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9899, Test Accuracy: 0.9901

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 2
Training 

In [13]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9919648927622227, 'test_acc': 0.9918888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9880709561777613, 'test_acc': 0.9882777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9891835094876074, 'test_acc': 0.9892777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9899870202113852, 'test_acc': 0.99}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9913468075900859, 'test_acc': 0.9913888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr'

In [14]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9919648927622227, 'test_acc': 0.9918888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9880709561777613, 'test_acc': 0.9882777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9891835094876074, 'test_acc': 0.9892777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9899870202113852, 'test_acc': 0.99}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9913468075900859, 'test_acc': 0.9913888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr'

In [4]:
baselines_results = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8666666666666667, 'test_acc': 0.8175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9162777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9333333333333333, 'test_acc': 0.8726666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8943888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.8821666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8978333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.8, 'test_acc': 0.8302777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.856}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.8666666666666667, 'test_acc': 0.8621666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.9089444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.8351666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9567222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.8339444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.9333333333333333, 'test_acc': 0.872}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.8, 'test_acc': 0.869}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.8983888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9001666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9125}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8578333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.9333333333333333, 'test_acc': 0.837}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9291111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.8337222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.8752777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8600555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9814814814814815, 'test_acc': 0.9613333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9444444444444444, 'test_acc': 0.9471666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9814814814814815, 'test_acc': 0.9660555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9938271604938271, 'test_acc': 0.9660555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9506172839506173, 'test_acc': 0.9515}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.9691358024691358, 'test_acc': 0.9482777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.8641975308641975, 'test_acc': 0.8548333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8395061728395061, 'test_acc': 0.8650555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.8765432098765432, 'test_acc': 0.8571666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.9629629629629629, 'test_acc': 0.9616666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.9691358024691358, 'test_acc': 0.9701666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9706666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9676666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.9753086419753086, 'test_acc': 0.9566666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.9629629629629629, 'test_acc': 0.9555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8765432098765432, 'test_acc': 0.8773888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9626666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9646666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.9629629629629629, 'test_acc': 0.9202777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.9814814814814815, 'test_acc': 0.9623888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.9814814814814815, 'test_acc': 0.9483888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.9814814814814815, 'test_acc': 0.9476666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9705555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.9629629629629629, 'test_acc': 0.9601111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.9320987654320988, 'test_acc': 0.9458333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9679012345679012, 'test_acc': 0.9652222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9691358024691358, 'test_acc': 0.9731666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.971604938271605, 'test_acc': 0.9736111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9753086419753086, 'test_acc': 0.9702222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.971604938271605, 'test_acc': 0.9661666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.9722222222222222, 'test_acc': 0.9745}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.8530864197530864, 'test_acc': 0.8548333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8518518518518519, 'test_acc': 0.8580555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.8777777777777778, 'test_acc': 0.8685555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.9709876543209877, 'test_acc': 0.9687222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.9592592592592593, 'test_acc': 0.9576111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.975925925925926, 'test_acc': 0.9718333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.8629629629629629, 'test_acc': 0.872}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.9814814814814815, 'test_acc': 0.9793333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.9722222222222222, 'test_acc': 0.9728333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8777777777777778, 'test_acc': 0.8707777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.9666666666666667, 'test_acc': 0.9721666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.9648148148148148, 'test_acc': 0.9685555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.921604938271605, 'test_acc': 0.9223333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.9814814814814815, 'test_acc': 0.9706666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9866666666666667, 'test_acc': 0.9861111111111112}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9879012345679012, 'test_acc': 0.9874444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.988641975308642, 'test_acc': 0.9868333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9835802469135803, 'test_acc': 0.9803888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9861728395061728, 'test_acc': 0.9870555555555556}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9919648927622227, 'test_acc': 0.9918888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9880709561777613, 'test_acc': 0.9882777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9891835094876074, 'test_acc': 0.9892777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9899870202113852, 'test_acc': 0.99}]
finetune1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8666666666666667, 'test_acc': 0.8175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9162777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9333333333333333, 'test_acc': 0.8726666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8943888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.8821666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8978333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.8, 'test_acc': 0.8302777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.856}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.8666666666666667, 'test_acc': 0.8621666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.9089444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.8351666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9567222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.8339444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.9333333333333333, 'test_acc': 0.872}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.8, 'test_acc': 0.869}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.8983888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9001666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9125}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8578333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.9333333333333333, 'test_acc': 0.837}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9291111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.8337222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.8752777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8600555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8666666666666667, 'test_acc': 0.8269444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9351666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9333333333333333, 'test_acc': 0.8673888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8336666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.9033888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8946111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.9101666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.7714444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.8666666666666667, 'test_acc': 0.8822222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.8522777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8645}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9601666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8607777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.9343333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.9333333333333333, 'test_acc': 0.9229444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.9256666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.8748888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9092222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8827777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.8778333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.9177777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.8452222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.8465555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.8168888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8942222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9333333333333333, 'test_acc': 0.8646111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9333333333333333, 'test_acc': 0.9331666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9068333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.9095}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.9094444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9325555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9333333333333333, 'test_acc': 0.9150555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.9191111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.9239444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.9067222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.8693888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9333333333333333, 'test_acc': 0.9282777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.8985555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.955}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.9526666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9333333333333333, 'test_acc': 0.9364444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9305}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9159444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.9274444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9112777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.9333333333333333, 'test_acc': 0.9263888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.9165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.9333333333333333, 'test_acc': 0.9243333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.9047777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.9333333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.8802777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9333333333333333, 'test_acc': 0.9478333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9428333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.9124444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.943}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9368333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8474444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.9426666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.9145}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.9302777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8666666666666667, 'test_acc': 0.8672777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9529444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9067777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.9316111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.9516111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9333333333333333, 'test_acc': 0.9148888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9281111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9338888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.942}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9367222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.9467777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.9460555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9441111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.9236666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.9488333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.8416111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9405}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9468333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.9158333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.9437777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9382222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8819444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.9523333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.9275555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.8905555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8857222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9585555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9128888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.9228888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.9411111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.9333333333333333, 'test_acc': 0.9289444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9403333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9244444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.9238888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9396666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.9267777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9385555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9137777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.9518333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.9471666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.8528333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9259444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.93}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8650555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.9453888888888888}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9213333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9333333333333333, 'test_acc': 0.9161666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.9438333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.9308333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.8721111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8671666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9307222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8921111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.9017222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9333333333333333, 'test_acc': 0.9437777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.9361666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9383888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9469444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9333333333333333, 'test_acc': 0.9256111111111112}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9141111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.9077222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9461666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9112777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.9118888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.9481666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.8936666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.9396666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9425555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8821666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.9218333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9467222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.9333333333333333, 'test_acc': 0.9443333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.9272777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.9352777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.8824444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8666666666666667, 'test_acc': 0.8275}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.9333333333333333, 'test_acc': 0.9287222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9270555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.8526666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.9481111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.9422777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9183333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9365}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.9319444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9266111111111112}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.9343333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9437222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9302777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.9060555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.9258888888888889}]
finetune2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9814814814814815, 'test_acc': 0.9613333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9444444444444444, 'test_acc': 0.9471666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9814814814814815, 'test_acc': 0.9660555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9938271604938271, 'test_acc': 0.9660555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9506172839506173, 'test_acc': 0.9515}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.9691358024691358, 'test_acc': 0.9482777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.8641975308641975, 'test_acc': 0.8548333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.8395061728395061, 'test_acc': 0.8650555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.8765432098765432, 'test_acc': 0.8571666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.9629629629629629, 'test_acc': 0.9616666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.9691358024691358, 'test_acc': 0.9701666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9706666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9676666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.9753086419753086, 'test_acc': 0.9566666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.9629629629629629, 'test_acc': 0.9555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.8765432098765432, 'test_acc': 0.8773888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9626666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.9646666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.9629629629629629, 'test_acc': 0.9202777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.9814814814814815, 'test_acc': 0.9623888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.9814814814814815, 'test_acc': 0.9483888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.9814814814814815, 'test_acc': 0.9476666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9705555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.9629629629629629, 'test_acc': 0.9601111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.9320987654320988, 'test_acc': 0.9458333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9814814814814815, 'test_acc': 0.9712777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9012345679012346, 'test_acc': 0.9083888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9012345679012346, 'test_acc': 0.8938888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9814814814814815, 'test_acc': 0.9626666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9629629629629629, 'test_acc': 0.9608888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.9814814814814815, 'test_acc': 0.9592777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9876543209876543, 'test_acc': 0.9626666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9938271604938271, 'test_acc': 0.9709444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9753086419753086, 'test_acc': 0.9661666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.9938271604938271, 'test_acc': 0.9702777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9074074074074074, 'test_acc': 0.9042222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9938271604938271, 'test_acc': 0.9666666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9876543209876543, 'test_acc': 0.9722222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.9693333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.9814814814814815, 'test_acc': 0.9657777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.9938271604938271, 'test_acc': 0.9697777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.8518518518518519, 'test_acc': 0.8632777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9629629629629629, 'test_acc': 0.9238333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.9753086419753086, 'test_acc': 0.9683888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.9814814814814815, 'test_acc': 0.9585}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.9382716049382716, 'test_acc': 0.8982222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.8888888888888888, 'test_acc': 0.8958888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.9320987654320988, 'test_acc': 0.9378888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.9814814814814815, 'test_acc': 0.9618888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.9135802469135802, 'test_acc': 0.9077777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9938271604938271, 'test_acc': 0.9683333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9506172839506173, 'test_acc': 0.9596666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9444444444444444, 'test_acc': 0.9255555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9753086419753086, 'test_acc': 0.9679444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9691358024691358, 'test_acc': 0.9643888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.9938271604938271, 'test_acc': 0.9755555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9691358024691358, 'test_acc': 0.9542777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.9938271604938271, 'test_acc': 0.9745}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9876543209876543, 'test_acc': 0.971}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.9876543209876543, 'test_acc': 0.9724444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.9876543209876543, 'test_acc': 0.9762777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9753086419753086, 'test_acc': 0.9535555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9876543209876543, 'test_acc': 0.9775555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9691358024691358, 'test_acc': 0.9626111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.9876543209876543, 'test_acc': 0.9676666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9938271604938271, 'test_acc': 0.9707222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.9876543209876543, 'test_acc': 0.9641666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9753086419753086, 'test_acc': 0.9215555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.9629629629629629, 'test_acc': 0.9493333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9506172839506173, 'test_acc': 0.9501111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.9753086419753086, 'test_acc': 0.9575}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9657222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.9629629629629629, 'test_acc': 0.9693888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.9629629629629629, 'test_acc': 0.9506666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.9938271604938271, 'test_acc': 0.9655}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9876543209876543, 'test_acc': 0.969}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9691358024691358, 'test_acc': 0.97}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9703888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9382716049382716, 'test_acc': 0.9587222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9691358024691358, 'test_acc': 0.9583333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.9876543209876543, 'test_acc': 0.9777777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9938271604938271, 'test_acc': 0.9772777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9197530864197531, 'test_acc': 0.9436666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9938271604938271, 'test_acc': 0.9751666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.9780555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.9382716049382716, 'test_acc': 0.9477777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.9814814814814815, 'test_acc': 0.9601111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9777222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.9691358024691358, 'test_acc': 0.962}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.9938271604938271, 'test_acc': 0.9777222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.9785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.9506172839506173, 'test_acc': 0.9465}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.9876543209876543, 'test_acc': 0.9488888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.9876543209876543, 'test_acc': 0.9715555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.9774444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.9691358024691358, 'test_acc': 0.95}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.9938271604938271, 'test_acc': 0.9705555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.9320987654320988, 'test_acc': 0.9621111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.9814814814814815, 'test_acc': 0.9652777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.9938271604938271, 'test_acc': 0.9627777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.9685}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9876543209876543, 'test_acc': 0.9733333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.9712222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9938271604938271, 'test_acc': 0.9710555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9753086419753086, 'test_acc': 0.9613333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.9814814814814815, 'test_acc': 0.9791111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.9938271604938271, 'test_acc': 0.9788888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.9727222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.9814814814814815, 'test_acc': 0.9781666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9444444444444444, 'test_acc': 0.9565}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.9644444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.9938271604938271, 'test_acc': 0.9773888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9753086419753086, 'test_acc': 0.9668333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.9876543209876543, 'test_acc': 0.9777222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.975}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9803333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9938271604938271, 'test_acc': 0.9539444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.9762222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.9938271604938271, 'test_acc': 0.9748888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.9753086419753086, 'test_acc': 0.9581666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.9938271604938271, 'test_acc': 0.9782777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.9753086419753086, 'test_acc': 0.9665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.9814814814814815, 'test_acc': 0.9638333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.9637222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9629629629629629, 'test_acc': 0.9530555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9876543209876543, 'test_acc': 0.9477222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9629629629629629, 'test_acc': 0.9650555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9629629629629629, 'test_acc': 0.9558888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.9818333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9197530864197531, 'test_acc': 0.9261666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9259259259259259, 'test_acc': 0.94}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.9876543209876543, 'test_acc': 0.9745555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9938271604938271, 'test_acc': 0.9812777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9876543209876543, 'test_acc': 0.9755}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9938271604938271, 'test_acc': 0.9665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.9825}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9629629629629629, 'test_acc': 0.9732777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9876543209876543, 'test_acc': 0.9802777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.9876543209876543, 'test_acc': 0.9677777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.9780555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.9938271604938271, 'test_acc': 0.9661666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9814814814814815, 'test_acc': 0.9646111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9753086419753086, 'test_acc': 0.9701666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.9876543209876543, 'test_acc': 0.9600555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.9938271604938271, 'test_acc': 0.9735}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.9815}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.9753086419753086, 'test_acc': 0.968}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.9938271604938271, 'test_acc': 0.9742222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.9604444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9444444444444444, 'test_acc': 0.9632777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9753086419753086, 'test_acc': 0.9542222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9876543209876543, 'test_acc': 0.968}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9506172839506173, 'test_acc': 0.9604444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.9876543209876543, 'test_acc': 0.9725}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.9691358024691358, 'test_acc': 0.9735555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.9938271604938271, 'test_acc': 0.9767777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.9753086419753086, 'test_acc': 0.9526111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.9938271604938271, 'test_acc': 0.9752777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.9876543209876543, 'test_acc': 0.9763333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.9814814814814815, 'test_acc': 0.9558888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.9938271604938271, 'test_acc': 0.9794444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.9938271604938271, 'test_acc': 0.9798333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.9382716049382716, 'test_acc': 0.9325}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.9938271604938271, 'test_acc': 0.9688333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.9629629629629629, 'test_acc': 0.9577222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.9814814814814815, 'test_acc': 0.9465555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.9876543209876543, 'test_acc': 0.9662222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.9938271604938271, 'test_acc': 0.9753888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.9691358024691358, 'test_acc': 0.9557777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.9752222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.98}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.9938271604938271, 'test_acc': 0.9575}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.9629629629629629, 'test_acc': 0.9695}]
finetune3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9679012345679012, 'test_acc': 0.9652222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9691358024691358, 'test_acc': 0.9731666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.971604938271605, 'test_acc': 0.9736111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9753086419753086, 'test_acc': 0.9702222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.971604938271605, 'test_acc': 0.9661666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.9722222222222222, 'test_acc': 0.9745}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.8530864197530864, 'test_acc': 0.8548333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.8518518518518519, 'test_acc': 0.8580555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.8777777777777778, 'test_acc': 0.8685555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.9709876543209877, 'test_acc': 0.9687222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.9592592592592593, 'test_acc': 0.9576111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.975925925925926, 'test_acc': 0.9718333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.8629629629629629, 'test_acc': 0.872}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.9814814814814815, 'test_acc': 0.9793333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.9722222222222222, 'test_acc': 0.9728333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.8777777777777778, 'test_acc': 0.8707777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.9666666666666667, 'test_acc': 0.9721666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.9648148148148148, 'test_acc': 0.9685555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.921604938271605, 'test_acc': 0.9223333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.9814814814814815, 'test_acc': 0.9706666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.980246913580247, 'test_acc': 0.9797777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9012345679012346, 'test_acc': 0.9003888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8864197530864197, 'test_acc': 0.8931666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9734567901234568, 'test_acc': 0.9731111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9765432098765432, 'test_acc': 0.9712777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.975925925925926, 'test_acc': 0.9777222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9808641975308642, 'test_acc': 0.9788888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.8925925925925926, 'test_acc': 0.8987222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9783950617283951, 'test_acc': 0.9778333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.9790123456790123, 'test_acc': 0.9740555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9666666666666667, 'test_acc': 0.9643888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9790123456790123, 'test_acc': 0.9689444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9592592592592593, 'test_acc': 0.9562777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.9790123456790123, 'test_acc': 0.9757222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.9660493827160493, 'test_acc': 0.9689444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.980246913580247, 'test_acc': 0.9763333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.858641975308642, 'test_acc': 0.8735555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9629629629629629, 'test_acc': 0.9699444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.9574074074074074, 'test_acc': 0.9558888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.9604938271604938, 'test_acc': 0.9488888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9753086419753086, 'test_acc': 0.9715555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9691358024691358, 'test_acc': 0.9677222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9814814814814815, 'test_acc': 0.9799444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9765432098765432, 'test_acc': 0.974}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9641975308641976, 'test_acc': 0.9583888888888888}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.987037037037037, 'test_acc': 0.9830555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.980246913580247, 'test_acc': 0.9792777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.9679012345679012, 'test_acc': 0.9732222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9388888888888889, 'test_acc': 0.9333333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.9728395061728395, 'test_acc': 0.9741666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.9777777777777777, 'test_acc': 0.9747777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9740740740740741, 'test_acc': 0.9678333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9845679012345679, 'test_acc': 0.9818888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9753086419753086, 'test_acc': 0.9751111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.9827160493827161, 'test_acc': 0.9805555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9777777777777777, 'test_acc': 0.9756111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.9771604938271605, 'test_acc': 0.9778888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9691358024691358, 'test_acc': 0.9707222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.975925925925926, 'test_acc': 0.9732777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9888888888888889, 'test_acc': 0.9791111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9864197530864197, 'test_acc': 0.9826111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9820987654320987, 'test_acc': 0.9842777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9888888888888889, 'test_acc': 0.9852222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9765432098765432, 'test_acc': 0.9771666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.975925925925926, 'test_acc': 0.9755}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.9858024691358025, 'test_acc': 0.9811111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9882716049382716, 'test_acc': 0.9856111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9814814814814815, 'test_acc': 0.9806666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9796296296296296, 'test_acc': 0.9773333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.9518518518518518, 'test_acc': 0.9575555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.9851851851851852, 'test_acc': 0.9805}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.9882716049382716, 'test_acc': 0.983}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.9641975308641976, 'test_acc': 0.9642777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.980246913580247, 'test_acc': 0.9810555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.9839506172839506, 'test_acc': 0.9850555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9882716049382716, 'test_acc': 0.9846666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.9858024691358025, 'test_acc': 0.9854444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.9462962962962963, 'test_acc': 0.9457777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.9901234567901235, 'test_acc': 0.9847777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.9888888888888889, 'test_acc': 0.9807222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9882716049382716, 'test_acc': 0.9819444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9827160493827161, 'test_acc': 0.9852777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9888888888888889, 'test_acc': 0.9859444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9876543209876543, 'test_acc': 0.9867777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.991358024691358, 'test_acc': 0.9857222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.9932098765432099, 'test_acc': 0.9849444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.9845679012345679, 'test_acc': 0.9852777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.9876543209876543, 'test_acc': 0.9858333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.9783950617283951, 'test_acc': 0.9811666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.9845679012345679, 'test_acc': 0.9826666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9864197530864197, 'test_acc': 0.9839444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.9901234567901235, 'test_acc': 0.9844444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.980246913580247, 'test_acc': 0.9792222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9845679012345679, 'test_acc': 0.9826111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.9845679012345679, 'test_acc': 0.9854444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.9808641975308642, 'test_acc': 0.9783888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.9833333333333333, 'test_acc': 0.9857777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9820987654320987, 'test_acc': 0.9849444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.9882716049382716, 'test_acc': 0.9857222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.9858024691358025, 'test_acc': 0.9776111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9796296296296296, 'test_acc': 0.9788888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9691358024691358, 'test_acc': 0.974}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9839506172839506, 'test_acc': 0.9832222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9783950617283951, 'test_acc': 0.9796666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9845679012345679, 'test_acc': 0.9817222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.9845679012345679, 'test_acc': 0.9838333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9851851851851852, 'test_acc': 0.9836111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9598765432098766, 'test_acc': 0.9644444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.975925925925926, 'test_acc': 0.9737222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9820987654320987, 'test_acc': 0.9836666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.991358024691358, 'test_acc': 0.9841666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9783950617283951, 'test_acc': 0.9807222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.9506172839506173, 'test_acc': 0.9588333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9734567901234568, 'test_acc': 0.974}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9746913580246913, 'test_acc': 0.9735}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.9827160493827161, 'test_acc': 0.9827222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.9629629629629629, 'test_acc': 0.9688888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.9746913580246913, 'test_acc': 0.9791111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9827160493827161, 'test_acc': 0.9835555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9734567901234568, 'test_acc': 0.9652777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9783950617283951, 'test_acc': 0.9770555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9648148148148148, 'test_acc': 0.9693888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9691358024691358, 'test_acc': 0.967}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9740740740740741, 'test_acc': 0.9715555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9777777777777777, 'test_acc': 0.9793333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.9753086419753086, 'test_acc': 0.9775}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.9629629629629629, 'test_acc': 0.9622777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.9783950617283951, 'test_acc': 0.9792777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.9783950617283951, 'test_acc': 0.9752222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.9765432098765432, 'test_acc': 0.9808888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.980246913580247, 'test_acc': 0.9742777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.980246913580247, 'test_acc': 0.9792777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.9876543209876543, 'test_acc': 0.9832222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.980246913580247, 'test_acc': 0.9825555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.9697530864197531, 'test_acc': 0.9698888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.9746913580246913, 'test_acc': 0.9746111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.9666666666666667, 'test_acc': 0.9711666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.9728395061728395, 'test_acc': 0.9727777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.9796296296296296, 'test_acc': 0.9812777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.9864197530864197, 'test_acc': 0.9772222222222222}]
finetune4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9866666666666667, 'test_acc': 0.9861111111111112}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9879012345679012, 'test_acc': 0.9874444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.988641975308642, 'test_acc': 0.9868333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9835802469135803, 'test_acc': 0.9803888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9861728395061728, 'test_acc': 0.9870555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9880246913580247, 'test_acc': 0.9882222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9830864197530864, 'test_acc': 0.9825}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9892592592592593, 'test_acc': 0.9871666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9885185185185185, 'test_acc': 0.9873888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9866666666666667, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9883950617283951, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9882716049382716, 'test_acc': 0.9877222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9879012345679012, 'test_acc': 0.9856666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9881481481481481, 'test_acc': 0.9864444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9879012345679012, 'test_acc': 0.9879444444444444}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9876543209876543, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9891358024691358, 'test_acc': 0.9884444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9907407407407407, 'test_acc': 0.9886666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9882716049382716, 'test_acc': 0.9878333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9859259259259259, 'test_acc': 0.9866666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9887654320987654, 'test_acc': 0.9882777777777778}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9875308641975309, 'test_acc': 0.987}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9895061728395061, 'test_acc': 0.9876111111111111}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9846913580246913, 'test_acc': 0.985}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9850617283950617, 'test_acc': 0.9865}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9860493827160494, 'test_acc': 0.9856666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9882716049382716, 'test_acc': 0.9882222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9867901234567901, 'test_acc': 0.9851111111111112}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9891358024691358, 'test_acc': 0.9886666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9853086419753087, 'test_acc': 0.9868888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9820987654320987, 'test_acc': 0.9822777777777778}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9822222222222222, 'test_acc': 0.9822777777777778}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.981358024691358, 'test_acc': 0.98}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9853086419753087, 'test_acc': 0.9851666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9829629629629629, 'test_acc': 0.9841111111111112}]
finetune5 = [{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9919648927622227, 'test_acc': 0.9918888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9880709561777613, 'test_acc': 0.9882777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9891835094876074, 'test_acc': 0.9892777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9899870202113852, 'test_acc': 0.99}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9913468075900859, 'test_acc': 0.9913888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9899252116941715, 'test_acc': 0.9901111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9906669139007355, 'test_acc': 0.9908333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9888126583843253, 'test_acc': 0.989}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9894925520736757, 'test_acc': 0.9896666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9922121268310773, 'test_acc': 0.9921666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9891217009703938, 'test_acc': 0.9892777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9896779776253167, 'test_acc': 0.9899444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9928302120032141, 'test_acc': 0.9926666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9893071265220348, 'test_acc': 0.9895555555555555}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9886272328326844, 'test_acc': 0.9889444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.988689041349898, 'test_acc': 0.9889444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9896779776253167, 'test_acc': 0.9898333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.988132764694975, 'test_acc': 0.9884444444444445}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9923975523827183, 'test_acc': 0.9924444444444445}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9876382965572655, 'test_acc': 0.9879444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9871438284195562, 'test_acc': 0.9874444444444445}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9870820199023426, 'test_acc': 0.9872777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9906669139007355, 'test_acc': 0.9908333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9898015946597442, 'test_acc': 0.9899444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9876382965572655, 'test_acc': 0.9877777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9880091476605476, 'test_acc': 0.9881111111111112}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9886272328326844, 'test_acc': 0.9887222222222222}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9886272328326844, 'test_acc': 0.9887222222222222}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9867729773162742, 'test_acc': 0.9869444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9872056369367699, 'test_acc': 0.9873888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9873910624884109, 'test_acc': 0.9874444444444445}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9886272328326844, 'test_acc': 0.9887777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9819519129736077, 'test_acc': 0.9820555555555556}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9830026577662402, 'test_acc': 0.9832777777777778}]

In [5]:
results = finetune1 + finetune2 + finetune3 + finetune4 + finetune5

In [22]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_random.json', 'w') as f:
    json.dump(results, f)
results = results[1:]

In [7]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_fashion_random.json', 'w') as f:
    json.dump(results, f)
results = results[1:]